In [2]:
import matplotlib.pyplot as plt
import tabulate

import numpy as np

import random

import pandas as pd

USERS = ["David", "Jeff", "Kay", "Sara", "Tana"]
METRICS = ["BLEU", "Rouge", "Mauve"]
MODEL_TYPES = ["FFT", "RoSA", "LoRA"]

COLUMNS = ["model", "user", "model_type", "metric", "RAFT", "RAG", "score", "metadata"]

In [3]:
def get_mock_results():
    df = pd.DataFrame(columns=COLUMNS)

    for user in USERS:
        for model_type in MODEL_TYPES:
            for metric in METRICS:
                for rag in [False, True]:
                    for raft in [False, True]:
                        df = df._append(
                            {
                                "model": "model",
                                "user": user,
                                "model_type": model_type,
                                "metric": metric,
                                "RAFT": raft,
                                "RAG": rag,
                                "score": random.random(),
                                "metadata": None,
                            },
                            ignore_index=True,
                        )

    return df


def create_all_results_table(results_df):
    table = []
    for model_type in MODEL_TYPES:
        for raft in [False, True]:
            for rag in [False, True]:
                # Create line for given model type and raft
                filtered_df = results_df[
                    (results_df["model_type"] == model_type)
                    & (results_df["RAFT"] == raft)
                    & (results_df["RAG"] == rag)
                ]
                raft_str = "-RAFT" if raft else ""
                rag_str = "-RAG" if rag else ""
                model_str = f"{model_type}{raft_str}{rag_str}"
                line = [model_str]
                for user in USERS:
                    for metric in METRICS:
                        try:
                            score = filtered_df[
                                (filtered_df["user"] == user) & (filtered_df["metric"] == metric)
                            ]["score"].values[0]
                        except Exception as e:
                            print(e)
                            score = -1
                        line.append(score)
                table.append(line)
    return table

mock_df = get_mock_results()
table = create_all_results_table(mock_df)
for line in table:
    print(line)

/tmp/ipykernel_2517939/2788884316.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append(


['FFT', 0.43943740079119564, 0.8668396962608216, 0.589313923576934, 0.5021292044151175, 0.6634976169442434, 0.354205851274626, 0.6210890716855332, 0.47279771656612546, 0.07885753784881433, 0.6166711053930242, 0.38031557249657455, 0.7903876565584012, 0.3854435604668386, 0.8423280956245982, 0.8723841623352212]
['FFT-RAG', 0.28350446773865723, 0.21005792426727443, 0.8091919225641164, 0.24440737639944388, 0.011639049902569476, 0.28223210320880343, 0.8248938744197795, 0.6158480525444884, 0.5918987079565855, 0.5808686123548831, 0.7214583548761658, 0.9336071828815578, 0.02712884369477775, 0.7282641022566084, 0.38561663666398704]
['FFT-RAFT', 0.3233458955710108, 0.21971098913324205, 0.3751505936316526, 0.7911277877120739, 0.3718870285326702, 0.7220792495346335, 0.8382005367698971, 0.008629614264878183, 0.4453693939018103, 0.3029337446930137, 0.1211460019717191, 0.20077792877935285, 0.36685052262394535, 0.2350105321879996, 0.5646940532735555]
['FFT-RAFT-RAG', 0.8351091222774694, 0.2876030016888

In [24]:
from pylatex import Document, Section, Tabular, MultiColumn, MultiRow, Command, NoEscape

def format_as_math(number):
    """Return number formatted as LaTeX math."""
    return NoEscape(f"${number}$")

def get_colored_column():
    """Return a colored column formatter."""
    return NoEscape(">{\columncolor{gray!10}}")

def create_document(results_table):
    doc = Document("results")

    columns_format = ""
    columns_format += "l"
    for i, user in enumerate(USERS):
        for metric in METRICS:
            if i % 2 == 0:
                columns_format += get_colored_column()
            columns_format += "c"
    # with doc.create(Tabular("c" * (1 + len(USERS) * len(METRICS)))) as table:
    with doc.create(Tabular(columns_format)) as table:
        table.add_hline()

        # Add user columns
        user_columns = []
        user_columns.append(" ")
        for i, user in enumerate(USERS):
            if False: #i % 2 == 0:
                user_columns.append(MultiColumn(3, align='c', data=Command('cellcolor', arguments=['gray!10'], extra_arguments=Command('texttt', user))))
            else:
                user_columns.append(MultiColumn(3, align='c', data=Command('texttt', user)))
        table.add_row(user_columns)
        table.add_hline()

        # Add metric columns
        metrics_columns = []
        metrics_columns.append("Method")
        for user in USERS:
            for metric in METRICS:
                metrics_columns.append(metric)
        table.add_row(metrics_columns)

        for i, line in enumerate(results_table, 1):
            clean_line = []
            for element in line:
                if isinstance(element, str):
                    element = Command('texttt', element)
                    clean_line.append(element)
                else:
                    element = float(element)
                    element = round(element, 3)
                    element = format_as_math(element)
                    clean_line.append(element)
            table.add_row(clean_line)

            # Add horizontal line after each model type
            if i % 4 == 0:
                table.add_hline()

        table.add_hline()

    return doc

doc = create_document(table)
print(doc.dumps())

\documentclass{article}%
\usepackage[T1]{fontenc}%
\usepackage[utf8]{inputenc}%
\usepackage{lmodern}%
\usepackage{textcomp}%
\usepackage{lastpage}%
%
%
%
\begin{document}%
\normalsize%
\begin{tabular}{l>{\columncolor{gray!10}}c>{\columncolor{gray!10}}c>{\columncolor{gray!10}}cccc>{\columncolor{gray!10}}c>{\columncolor{gray!10}}c>{\columncolor{gray!10}}cccc>{\columncolor{gray!10}}c>{\columncolor{gray!10}}c>{\columncolor{gray!10}}c}%
\hline%
 &\multicolumn{3}{c}{\texttt{David}}&\multicolumn{3}{c}{\texttt{Jeff}}&\multicolumn{3}{c}{\texttt{Kay}}&\multicolumn{3}{c}{\texttt{Sara}}&\multicolumn{3}{c}{\texttt{Tana}}\\%
\hline%
Method&BLEU&Rouge&Mauve&BLEU&Rouge&Mauve&BLEU&Rouge&Mauve&BLEU&Rouge&Mauve&BLEU&Rouge&Mauve\\%
\texttt{FFT}&$0.439$&$0.867$&$0.589$&$0.502$&$0.663$&$0.354$&$0.621$&$0.473$&$0.079$&$0.617$&$0.38$&$0.79$&$0.385$&$0.842$&$0.872$\\%
\texttt{FFT{-}RAG}&$0.284$&$0.21$&$0.809$&$0.244$&$0.012$&$0.282$&$0.825$&$0.616$&$0.592$&$0.581$&$0.721$&$0.934$&$0.027$&$0.728$&$0.386$\\%
\te